In [1]:
#import Library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import cv2
import pydot

In [3]:
#ubah tensorflow menggunakan mode GPU
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
else:
  print("No GPU device found")

#GPU CUDA tidak dapat digunakan

No GPU device found


In [4]:
#fungsi load_dataset mengambil gambar dari directory
#setelah gambar diambil selanjutnya gambar dipisahkan berdasarkan labelnya(nama directory)

def load_dataset(directory):
  images = []
  labels = []
  for idx, label in enumerate(uniq_labels):
    for file in os.listdir(directory + '/'+label):
      filepath = directory +'/'+ label + "/" + file
      img = cv2.resize(cv2.imread(filepath),(50,50)) #gambar yang diambil diresize menjadi ukuran 50x50 untuk memudahkan komputasi
      images.append(img)
      labels.append(idx)
  #append gambar ke dalam list dengan bentuk array agar bisa dilakukan perhitungan
  images = np.asarray(images)
  labels = np.asarray(labels)
  return images, labels

In [5]:
#display_images function to show examples
def display_images(x_data,y_data, title, display_label = True):
    x, y = x_data,y_data
    fig, axes = plt.subplots(5, 8, figsize = (18, 5))
    fig.subplots_adjust(hspace = 0.5, wspace = 0.5)
    fig.suptitle(title, fontsize = 18)
    for i, ax in enumerate(axes.flat):
        ax.imshow(cv2.cvtColor(x[i], cv2.COLOR_BGR2RGB))
        if display_label:
            ax.set_xlabel(uniq_labels[y[i]])
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()

In [6]:
#pengambilan data dari directory menggunakan fungsi yang telah didefinisikan sebelumnya
data_dir = r'D:\SEMESTER 5\COMPUTER VISION\Project Compvis\Gesture Image Data'
uniq_labels = sorted(os.listdir(data_dir))
X_pre, Y_pre = load_dataset(data_dir)
print(X_pre.shape, Y_pre.shape)

(55500, 50, 50, 3) (55500,)


In [7]:
#split data untuk train, test, dan validation
X_train, X_test, Y_train, Y_test = train_test_split(X_pre, Y_pre, test_size = 0.8)
X_test, X_eval, Y_test, Y_eval = train_test_split(X_test, Y_test, test_size = 0.5)

In [9]:
#print untuk melihat shape untuk masing - masing data train, test, dan validation
#serta melihat label yang diikutkan apakah sesuai dengan data yang ada
print("Train images shape",X_train.shape, Y_train.shape)
print("Test images shape",X_test.shape, Y_test.shape)
print("Evaluate image shape",X_eval.shape, Y_eval.shape)
print("Printing the labels",uniq_labels, len(uniq_labels))
display_images(X_train,Y_train,'Samples from Train Set')
display_images(X_test,Y_test,'Samples from Test Set')
display_images(X_eval,Y_eval,'Samples from Validation Set')

Train images shape (11100, 50, 50, 3) (11100,)
Test images shape (22200, 50, 50, 3) (22200,)
Evaluate image shape (22200, 50, 50, 3) (22200,)
Printing the labels ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_'] 37


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
# mengubah data label menggunakan one hot encoding untuk memudahkan proses modelling
# contoh dari one hot encoding => '1' is represented as [0. 1. 0. . . . . 0.]
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)
Y_eval = to_categorical(Y_eval)

#normalisasi gambar, menggunakan max pixel value yaitu 255 sehingga nilai gambar menjadi 0-1
X_train = X_train / 255.
X_test = X_test/ 255.
X_eval = X_eval/ 255.

In [22]:
# Build model
model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation ='relu', input_shape=(50,50,3)),
        tf.keras.layers.Conv2D(16, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(16, (3,3), activation ='relu'),
        tf.keras.layers.MaxPool2D((2,2)),
        tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(32, (3,3), activation ='relu'),
        tf.keras.layers.MaxPool2D((2,2)),
        tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
        tf.keras.layers.Conv2D(64, (3,3), activation ='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(37, activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 48, 48, 16)        448       
                                                                 
 conv2d_10 (Conv2D)          (None, 46, 46, 16)        2320      
                                                                 
 conv2d_11 (Conv2D)          (None, 44, 44, 16)        2320      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 22, 22, 16)       0         
 2D)                                                             
                                                                 
 conv2d_12 (Conv2D)          (None, 20, 20, 32)        4640      
                                                                 
 conv2d_13 (Conv2D)          (None, 18, 18, 32)        9248      
                                                      

In [23]:
#compile model
#optimizer adam
#menggunakan batch size dan learning rate default yaitu 32 (batch size) dan 0.001 (learning rate)
model.compile(optimizer = 'adam',
               loss = 'categorical_crossentropy', #loss function categorical crossentropy karena label terdiri dari 37 label berbeda
               metrics=['accuracy'],)

In [24]:
#melakukan fit pada data (melatih data untuk mendapatkan model)
history = model.fit(X_train, Y_train, epochs=20, verbose=1,
                validation_data=(X_eval, Y_eval))

Epoch 1/20
347/347 [==============================] - 47s 132ms/step - loss: 0.8510 - accuracy: 0.7591 - val_loss: 0.1906 - val_accuracy: 0.9336
Epoch 2/20
347/347 [==============================] - 44s 127ms/step - loss: 0.1281 - accuracy: 0.9589 - val_loss: 0.0975 - val_accuracy: 0.9699
Epoch 3/20
347/347 [==============================] - 47s 135ms/step - loss: 0.0760 - accuracy: 0.9766 - val_loss: 0.0627 - val_accuracy: 0.9845
Epoch 4/20
347/347 [==============================] - 44s 128ms/step - loss: 0.0408 - accuracy: 0.9873 - val_loss: 0.0241 - val_accuracy: 0.9926
Epoch 5/20
347/347 [==============================] - 44s 127ms/step - loss: 0.0255 - accuracy: 0.9926 - val_loss: 0.0766 - val_accuracy: 0.9775
Epoch 6/20
347/347 [==============================] - 46s 133ms/step - loss: 0.0317 - accuracy: 0.9904 - val_loss: 0.0558 - val_accuracy: 0.9821
Epoch 7/20
347/347 [==============================] - 45s 130ms/step - loss: 0.0459 - accuracy: 0.9871 - val_loss: 0.0369 - val_ac

In [25]:
#testing
model.evaluate(X_test, Y_test)

694/694 [==============================] - 12s 18ms/step - loss: 0.0017 - accuracy: 0.9996


[0.0017344903899356723, 0.9995945692062378]

In [26]:
#save the model
model.save(r'D:\SEMESTER 5\COMPUTER VISION\Project Compvis\model_sign-language_kel5.h5')

In [27]:
train_loss = history.history['loss']
train_acc = history.history['accuracy']
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

In [34]:
print(len(train_loss))
print(len(train_acc))
print(len(val_loss))
print(len(val_accuracy))

20
20
20
20


In [ ]:
#ploting training and validation loss vs. epochs

epochs = list(range(1,21))
plt.plot(epochs, train_loss, label = "training loss")
plt.plot(epochs, val_loss, label = "validation  loss")
plt.legend()
plt.show()

In [50]:
#ploting training and validation accuracy vs. epochs


plt.plot(epochs, train_acc, label = "training accuracy")
plt.plot(epochs, val_accuracy, label = "validation  accuracy")
plt.legend()
plt.show()